In [ ]:
from flask import Flask, render_template, request
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
import MySQLdb




######### Initialise Flaks App##########

app = Flask(__name__)
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0

######### MySQL Database Setup ##########
#connection = MySQLdb.connect(host="realkori330.mysql.pythonanywhere-services.com",
#                     user="realkori330",
#                     passwd="melonztg960103",
#                     db="realkori330$default",
#                     charset='utf8mb4')

######### Login Manager Setup ##########

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "melonZTG960103!"


@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

############ Web Page Routes Setup ###############

@app.route("/")
@login_required
def home():
    return render_template('home2.html')

@app.route("/login")
def login():
    message = '請先登錄.'
    return render_template('login.html', message=message)

@app.route('/query_auth')
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "korizhai@163.com" and password == "0330":
        login_user(User(1))
        message = "Dear " + username + ", 您已被授權，歡迎來到犬種百科."
        return make_response(message, 200, )
    else:
        message = '密鑰錯誤!'
        return make_response( messagte,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "korizhai@163.com" and password == "0330":
        login_user(User(1))
        message = "Dear " + username + ", 您已被授權，歡迎來到犬種百科."
        return render_template('home2.html', message=message)
    else:
        message = '密鑰錯誤!'
        return render_template('login.html',message=message)

@app.route('/album')
@login_required
def album():
    connection = MySQLdb.connect(host="realkori330.mysql.pythonanywhere-services.com",
                     user="realkori330",
                     passwd="melonztg960103",
                     db="realkori330$default",
                     charset='utf8mb4')
    user = {"name":"Kori"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM dog_type"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['dog_type'] = i[0]
        dict['image_link'] = i[1]
        dict['size'] = i[2]
        dict['function'] = i[3]
        dict['specialty'] = i[4]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    cursor.close()
    connection.close()
    return render_template('index3.html',album_user=user, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = '謝謝 您已登出'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = '密鑰錯誤！'
    return render_template('login.html',message=message),500

######### API Endpoints ##########

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "korizhai@163.com" and password == "0330":
        login_user(User(1))
        message = "Dear " + username + ",, 您已被授權，歡迎來到犬種百科"
        return jsonify({'response':'ok!','message':message})
    else:
        message = '密鑰錯誤！'
        return jsonify({'response':'無效','message':'信息有誤'})

@app.route('/api_album', methods=['POST'])
@login_required
def api_album():
    user = {"name":"Kori"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM dog_type"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['dog_type'] = i[0]
        dict['image_link'] = i[1]
        dict['size'] = i[2]
        dict['function'] = i[3]
        dict['specialty'] = i[4]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return jsonify({'Album': dataset})

if __name__ == '__main__':
   app.run(debug = True)
